# Network Analysis with Python and the NetworkX package

In [117]:
import networkx as nx
import csv
from operator import itemgetter
import community
# import the packages we need for doing network analysis and for working with structured data

In [118]:
with open('quakers_nodelist.csv', 'r') as nodecsv:
    nodereader = csv.reader(nodecsv)
    nodes = [n for n in nodereader][1:]
# read in the csv-file quakers_nodelist and store the information in a variable called nodereader

In [119]:
node_names = [n[0] for n in nodes]
# get a list of just the node names (the first item in each row)

In [120]:
# print(node_names)

In [121]:
with open('quakers_edgelist.csv', 'r') as edgecsv:
    edgereader = csv.reader(edgecsv)
    edges = [tuple(e) for e in edgereader][1:]
# Read in the edgelist file

In [122]:
print(len(node_names))

119


In [123]:
print(len(edges))

174


In [124]:
G = nx.Graph()
# Initialize the graph object

In [125]:
G.add_nodes_from(node_names)
# Add nodes to the graph

In [126]:
G.add_edges_from(edges)
# Add edges to the graph

In [127]:
print(nx.info(G))
# Print information about the graph

Name: 
Type: Graph
Number of nodes: 119
Number of edges: 174
Average degree:   2.9244


## Attributes

In [128]:
hist_sig_dict = {}
gender_dict = {}
birth_dict = {}
death_dict = {}
id_dict = {}
# Create dictionaries for each column in the nodes list; they are empty {} 
# and will be filled later with contents from the quaker_nodes.csv

In [129]:
for node in nodes:
    hist_sig_dict[node[0]] = node[1]
    gender_dict[node[0]] = node[2]
    birth_dict[node[0]] = node[3]
    death_dict[node[0]] = node[4]
    id_dict[node[0]] = node[5]
# Loop through the list, one row at a time
# We create 'pairs' where we assign the attributes on indexes[1,2,3,4,5] to the node on index[0]
# Think about it like this:
# In row 1, the first name is 'Josep Wyeth'
# Joseph has a couple of attributes that describe him in a way that is useful for our research
# For example, Joseph was a religious writer, an attribute from the column 'Historical Significance'.
# Think about the column as a dictionary in Python terminology (stuff that is between {})
# Joseph is a man, another attribute, this time from the 3rd row (in Python counting, index[2];
# the information is stored in our dictionary called 'gender_dict'
# And so on
# For every one of Joseph's attributes we have to tell NetworkX to pair it with the node Joseph
# A node can have any number of attributes (as far as I know)

In [130]:
nx.set_node_attributes(G, hist_sig_dict, 'historical_significance')
nx.set_node_attributes(G, gender_dict, 'gender')
nx.set_node_attributes(G, birth_dict, 'birthyear')
nx.set_node_attributes(G, death_dict, 'deathyear')
nx.set_node_attributes(G, id_dict, 'id')
# Add attributes to the graph G; set_node_attribute takes three variables:
# 1) the graph G to which you are adding the attribute
# 2) the dictionary of id-attribute pairs
# 3) the name of the new attribute
# The result is that all nodes now have the six attributes and we can access this information any time

In [131]:
# Example: Print out all birth years of the nodes
for n in G.nodes():
    print(n, G.node[n]['gender'])

Joseph Wyeth male
Alexander Skene of Newtyle male
James Logan male
Dorcas Erbery female
Lilias Skene male
William Mucklow male
Thomas Salthouse male
William Dewsbury male
John Audland male
Richard Claridge male
William Bradford male
Fettiplace Bellers male
John Bellers male
Isabel Yeamans female
George Fox the younger male
George Fox male
John Stubbs male
Anne Camm female
John Camm male
Thomas Camm male
Katharine Evans female
Lydia Lancaster female
Samuel Clarridge male
Thomas Lower male
Gervase Benson male
Stephen Crisp male
James Claypoole male
Thomas Holme male
John Freame male
John Swinton male
William Mead male
Henry Pickworth male
John Crook male
Gilbert Latey male
Ellis Hookes male
Joseph Besse male
James Nayler male
Elizabeth Hooten female
George Whitehead male
John Whitehead male
William Crouch male
Benjamin Furly male
Silvanus Bevan male
Robert Rich male
John Whiting male
Christopher Taylor male
Thomas Lawson male
Richard Farnworth male
William Coddington male
Thomas Taylor m

In [132]:
# Now lets do that for the other attributes, too!

# Network Analysis

In [133]:
# Now you have learned how to create a Graph object and how to add attributes to it.
# In the next section we will learn a variety of metrics available in NetworkX and how to access them.
# That was the most complex and dense code you will write in this workshop!

## Density

In [134]:
# Network Density
# The ratio of actual edges in the network to all possible edges in the network; in an undirected network,
# there could be a single edge between any two nodes, but as you can see in the visualisation, only a few
# of those possible edges are actually present.
# Network density gives you a quick sense of how closely knit your network is

In [135]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.02478279447372169


In [136]:
# Finding the shortest path between two nodes
# Margaret Fell and George Whitehead
fell_whitehead_path = nx.shortest_path(G, source='Margaret Fell', target='George Whitehead')
print('Shortest path between Fell and Whitehead:', fell_whitehead_path)

Shortest path between Fell and Whitehead: ['Margaret Fell', 'George Fox', 'George Whitehead']


In [137]:
# This function shows us the shortest path between Fell and Whitehead is one George Fox;
# Fell and Whitehead are not directly connected, but indirectly via a mutual contact

In [138]:
print('Length of that path:', len(fell_whitehead_path)-1)

Length of that path: 2


## Diameter

In [139]:
# Diameter
# Is the longest of all short paths; this measure is designed to give you a sense of the network's overall size,
# the distance from one end of the network to another
# Using the function nx.diameter(G) will result in an error that tells us that it is not possible calculate the
# diameter because the graph is not connected. Which means, that there are nodes in our graph that are only 
# connected to each other, but not to any other nodes.
# So, lets check for that first!

In [140]:
print(nx.is_connected(G))
# Returs False if the graph is not connected

False


In [141]:
components = nx.connected_components(G)
largest_component = max(components, key=len)
# Using nx.connected_components to get the list of components,
# then using the max(f) function to find the largest one
# From this we create a subgraph of just the largest component,
# then calculate the diameter of this subgraph, like with density:
subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print('Network diameter of largest component:', diameter)

Network diameter of largest component: 8


In [142]:
# Triadic closure
triadic_closure = nx.transitivity(G)
print('Triadic closure:', triadic_closure)

Triadic closure: 0.16937799043062202


## Centrality

In [143]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
print(G.node['William Penn'])
# This function adds degree, which is has calculated from the graph, as an attribute to the nodes.
# In one go we create a now dictionary (think of it as another column) called degree where we store
# the results of this calculation for each node
# With the print command we can check if we were successful adding the degree-attribute

{'historical_significance': 'Quaker leader and founder of Pennsylvania', 'gender': 'male', 'birthyear': '1644', 'deathyear': '1718', 'id': '10009531', 'degree': 18}


In [144]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print('Top 20 nodes by degree:')
for d in sorted_degree[:20]:
    print(d)

Top 20 nodes by degree:
('George Fox', 22)
('William Penn', 18)
('James Nayler', 16)
('George Whitehead', 13)
('Margaret Fell', 13)
('Benjamin Furly', 10)
('Edward Burrough', 9)
('George Keith', 8)
('Thomas Ellwood', 8)
('Francis Howgill', 7)
('John Perrot', 7)
('John Audland', 6)
('Richard Farnworth', 6)
('Alexander Parker', 6)
('John Story', 6)
('John Stubbs', 5)
('Thomas Curtis', 5)
('John Wilkinson', 5)
('William Caton', 5)
('Anthony Pearson', 5)


In [145]:
# Lets try this for a couple of other sortings
# For example we can change the Top 20 to a Top 5 or a Top 30
# Keep in mind that lists with more than 20 items are hard to 'quickly get' for humans
# We can also sort from the lowest degree to the hightest
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print('Lowest 15 nodes by degree:')
for d in sorted_degree[104:]:
    print(d)

Lowest 15 nodes by degree:
('Gideon Wanton', 1)
('John Wanton', 1)
('Grace Chamber', 1)
('Edward Haistwell', 1)
('John ap John', 1)
('John Rous', 1)
('Sarah Cheevers', 1)
('John Penington', 1)
('Humphrey Woolrich', 1)
('Mary Pennyman', 1)
('Dorothy Waugh', 1)
('Lewis Morris', 1)
('Thomas Story', 1)
('William Simpson', 1)
('Samuel Bownas', 1)


In [146]:
# Two more centrality measures, betweenness and eigenvector
betweenness_dict = nx.betweenness_centrality(G)
eigenvector_dict = nx.eigenvector_centrality(G)
# Now we assign each as an attribute to the nodes in our network:
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
# Now lets sort our nodes by these attributes as we have done before, only showing the Top 20

In [147]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)
print('Top 20 nodes by betweenness factor:')
for b in sorted_betweenness[:20]:
    print(b)

Top 20 nodes by betweenness factor:
('William Penn', 0.23999456006192205)
('George Fox', 0.23683257726065216)
('George Whitehead', 0.12632024847366005)
('Margaret Fell', 0.12106792237170329)
('James Nayler', 0.10446026280446098)
('Benjamin Furly', 0.06419626175167242)
('Thomas Ellwood', 0.046190623885104545)
('George Keith', 0.045006564009171565)
('John Audland', 0.04164936340077581)
('Alexander Parker', 0.03893676140525336)
('John Story', 0.028990098622866983)
('John Burnyeat', 0.028974117533439564)
('John Perrot', 0.02829566854990583)
('James Logan', 0.026944806605823553)
('Richard Claridge', 0.026944806605823553)
('Robert Barclay', 0.026944806605823553)
('Elizabeth Leavens', 0.026944806605823553)
('Thomas Curtis', 0.026729751729751724)
('John Stubbs', 0.024316593960227152)
('Mary Penington', 0.02420824624214454)


In [148]:
sorted_eigenvector = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)
print('Top 20 nodes by eigenvector:')
for e in sorted_eigenvector[:20]:
    print(e)

Top 20 nodes by eigenvector:
('George Fox', 0.4491750710859924)
('James Nayler', 0.3352974100447867)
('William Penn', 0.2703220115399868)
('Margaret Fell', 0.253170949905681)
('George Whitehead', 0.2497455334914196)
('Edward Burrough', 0.23147427604862297)
('Francis Howgill', 0.1909539378268105)
('Benjamin Furly', 0.1878520634691651)
('John Perrot', 0.1849692807795611)
('George Keith', 0.18384690867915351)
('Thomas Ellwood', 0.17608142535843857)
('Richard Farnworth', 0.15368535029296415)
('John Crook', 0.1327158126880779)
('Rebecca Travers', 0.1184804064465093)
('Alexander Parker', 0.11587808682088324)
('Anthony Pearson', 0.11120476725256784)
('William Dewsbury', 0.11057869321157121)
('John Stubbs', 0.10693500692141825)
('John Audland', 0.0983088971933375)
('Thomas Salthouse', 0.09548628544138771)


In [149]:
# Detecting interesting stuff in our network but finding out what's unusual or unexpected
# We can do this by comparing degree and betweenness
# Lets look at Elizabeth Leavens, who has a high betweenness factor, but doesn't show up in the high degree list
# For this, we get first the Top 20 betweenness nodes as a list
top_betweenness = sorted_betweenness[:20]
# Then we find and print their degree:
for tb in top_betweenness:
    degree = degree_dict[tb[0]]
    print('Name:', tb[0], '| Betweenness Centrality:', tb[1], '| Degree:', degree)

Name: William Penn | Betweenness Centrality: 0.23999456006192205 | Degree: 18
Name: George Fox | Betweenness Centrality: 0.23683257726065216 | Degree: 22
Name: George Whitehead | Betweenness Centrality: 0.12632024847366005 | Degree: 13
Name: Margaret Fell | Betweenness Centrality: 0.12106792237170329 | Degree: 13
Name: James Nayler | Betweenness Centrality: 0.10446026280446098 | Degree: 16
Name: Benjamin Furly | Betweenness Centrality: 0.06419626175167242 | Degree: 10
Name: Thomas Ellwood | Betweenness Centrality: 0.046190623885104545 | Degree: 8
Name: George Keith | Betweenness Centrality: 0.045006564009171565 | Degree: 8
Name: John Audland | Betweenness Centrality: 0.04164936340077581 | Degree: 6
Name: Alexander Parker | Betweenness Centrality: 0.03893676140525336 | Degree: 6
Name: John Story | Betweenness Centrality: 0.028990098622866983 | Degree: 6
Name: John Burnyeat | Betweenness Centrality: 0.028974117533439564 | Degree: 4
Name: John Perrot | Betweenness Centrality: 0.0282956685

## Community Detection with Modularity

In [150]:
# Commonly, what we want to know about networks is if there are subgroups or communities within the larger social structure
# Is it a network where everyone knows everyone? Or is several subgroups that are only connected by a few, or just one,
# person?
# There are many ways to calculate communities, but most commonly used is Modularity.
# Modularity is a measure of relative density in the network; A community, or 'module' has high density relative
# to other nodes within its community or module but low density with those outside. Modularity gives an overall
# score of how fractious a network is. That score can be used to partition the network and return the individual communities.

In [151]:
# Community detection with NetworkX requires a little more setup than other metrics
# Modularity is not included in the NetworkX package, but there is an additional package for this, the 
# python-louvain packgage we downloaded in the beginning
# For community detection we only want one function from it, best_partition()

In [153]:
# import community
# communities = community.best_partition(G)

# OBS! This produces an AttributeError, see Issue#3 here https://github.com/taynaud/python-louvain/issues/3
# Following the suggestions, I have not succeeded in getting it running yet

In [154]:
#nx.set_node_attributes(G, communities, 'modularity')